In [1]:
import json
import os
import re

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from nltk import download
from nltk.tokenize import word_tokenize
import torch
from tqdm.auto import tqdm
from transformers import MBart50Tokenizer, MBartForConditionalGeneration
from transformers import pipeline

download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Konstruktikon:

In [2]:
def parse_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')
    try:
        category, name = root.attrib.get('name').split(':')
    except ValueError:
        category = root.attrib.get('name')
        name = None

    sentences_data = []
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        contextleft = sentence.find('.//contextleft').text.strip()
        contextright = sentence.find('.//contextright').text.strip()
        
        sentences_data.append({
            'uid': uid,
            'text': text,
            'contextleft': contextleft,
            'contextright': contextright,
            'construction_id': int(construction_id),
            'category': category,
            'name': name,
        })

    return sentences_data

kee_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_sentences(xml_file)
        if data:
            kee_list += data

sentences = pd.DataFrame.from_dict(kee_list)
sentences.set_index('uid', inplace=True)
sentences

,text,contextleft,contextright,construction_id,category,name
uid,,,,,,
57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,Und dann ist da noch das generelle Problem mit...,"""""Wir haben die Gaza-Offensive nicht begonnen,...",Der ehemalige Ministerpräsident Hanija gehört ...,10,Negation,NEG_X_geschweige_denn_Y
D86B38CFA5D2458D9F3615EDD55C9308DF423DB0,»Ohne Europa sind viele Fragen nicht mehr seri...,Die Zeit nationaler Alleingänge sei längst vor...,"Die Frage ist: Kann es Europa gelingen, sich i...",10,Negation,NEG_X_geschweige_denn_Y
7B3FAB8B01ED862D3D3FA18531B0904BCF24DFF0,"Dies lässt sich damit begründen , dass vor und...","Wenn Alexander das Heer nicht geführt hätte, d...",Ganz anders ist die Beweislage für den ersten ...,10,Negation,NEG_X_geschweige_denn_Y
B003E526D0AA301DA46458D32A82CEEFD7ADADA5,"Dennoch hat Hamas es nicht geschafft , ihre mi...",DerCouncil of Foreign Relations schätzt das jä...,"Es ist in erster Linie die politische Führung,...",10,Negation,NEG_X_geschweige_denn_Y
C3FFF1CA55CA8FF0A806C33E14A0FB331582698E,"Abermillionen rings um die Welt , die das Spek...",Nichts hätte die Mär vom Niedergang der Superm...,"Doch wie schon die Wahl Obamas global war, ist...",10,Negation,NEG_X_geschweige_denn_Y
...,...,...,...,...,...,...
B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,""" "" Aber da wird geredet und geredet und , sob...",Zuerst hat er Flirtportale im Netz ausprobiert.,"Also geht er, der Computerfreak, lieber den al...",993,Eigenschaft,Intransitiv_passiv
32E879397C3BA674F547058D7CB2B781C55FE1AC,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...","Das ist ja das Besondere, wenn man so ein Turn...",Das taten die Spielerinnen schon im legendären...,993,Eigenschaft,Intransitiv_passiv
F0F280AA87B32AF6C4122E5B9C7906A31E88E555,"Hier wird getankt , und erst in der Morgenfrüh...",Schließlich schafft es der Wagen wieder auf di...,Von dort geht es gleich ins Berchtesgadener La...,993,Eigenschaft,Intransitiv_passiv


In [3]:
constructions = sentences[['construction_id', 'category', 'name']].drop_duplicates()
constructions.set_index('construction_id', inplace=True)
constructions

,category,name
construction_id,,
10,Negation,NEG_X_geschweige_denn_Y
100,Äquativ_Plural,gleich_ADJ
1004,Superlativ_Klimax,ADJ1_ADJ1-er_NP
1006,Superlativ,PRÄP_ADJ-ster_NP
101,Äquativ,ADJ_wie_NP
...,...,...
98,Äquativ,so_ADJ_wie_XP
980,Quantifizierung,CARD_KATEGORIE_NP
985,Relation_zweistellig,Transitiv_aktiv


In [4]:
constructions['category'].value_counts()

category
Quantifizierung                     14
Intensivierung                      14
Kategorisierung                     13
Exklamativ                          13
Äquativ                             12
                                    ..
Reflexives_Partikelverb              1
Reflexiver_Weg                       1
Konkretisierung_exhaustiv            1
Konkretisierung_exemplifizierend     1
Korrelation_Affirmation              1
Name: count, Length: 181, dtype: int64

In [5]:
def parse_kee(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')

    ke_data = []
    
    for ke in root.findall('.//ke'):
        ke_data.append({
            'ke_id': int(ke.attrib.get("id")),
            'ke_name': ke.attrib.get("name"),
            'ke_coretype': ke.attrib.get("coretype"),
            'ke_attributes': ke.attrib.get("attributes")
        })

    kee_data = []
    
    for kee in root.findall('.//kee'):
        for ke in ke_data:
            kee_data.append({
                'kee_id': int(kee.attrib.get("id")),
                'kee_name': kee.attrib.get("name"),
                'construction_id': int(construction_id)
            } | ke)

    return kee_data

kee_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_kee(xml_file)
        if data:
            kee_list += data

kees = pd.DataFrame.from_dict(kee_list)
kees.set_index('kee_id', inplace=True)
kees

,kee_name,construction_id,ke_id,ke_name,ke_coretype,ke_attributes
kee_id,,,,,,
1,geschweige_denn,10,1,Fokuskontext,CORE,
1,geschweige_denn,10,2,Erstes_Konjunkt,CORE,
1,geschweige_denn,10,3,Zweites_Konjunkt,CORE,[]
1,geschweige_denn,10,2350,Negator,CORE,
1,geschweige_denn,10,1,Fokuskontext,None,None
...,...,...,...,...,...,...
643,er,99,336,Dimension,CORE,
643,er,99,337,Bezugswert,CORE,
643,er,99,338,Quantifizierung,NONCORE,


In [6]:
kees['ke_name'].value_counts()

ke_name
Verglichenes        108
Dimension            78
Bezugskontext        78
Bezugswert           77
Antezedens           64
                   ... 
Zugeordnetes          1
Von_X_Geäußertes      1
Sprecher_X            1
Sprecher              1
Direktiv              1
Name: count, Length: 247, dtype: int64

In [7]:
kees['kee_name'].value_counts()

kee_name
(_)                      75
Klammern                 52
von                      41
sein                     38
und                      36
                         ..
par_excellence            1
wie_es_im_Buche_steht     1
zu_Tode                   1
wie_viel_Uhr              1
nicht_un-                 1
Name: count, Length: 372, dtype: int64

Die KEEs sind markiert wie folgt:

```
            <annotationset>
            ...
                <layer name="KEE-Negation:NEG_X_geschweige_denn_Y" descriptor="KEE-Negation:NEG_X_geschweige_denn_Y">
                    <label start="72" end="87" name="geschweige_denn" refid="GC_KEE:1" itype="null" groupid="0"></label>
                </layer>
            ...
            </annotationset>
```

In [8]:
def parse_masked_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    sentences_data = []
    
    constr_id = int(root.attrib.get('id'))
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        
        kees = []
        kees_idx = []
        kes = []
        kes_idx = []
        
        # Loop through annotations:
        for layer in sentence.findall('.//layer'):
            layer_name = layer.attrib.get('name')
            
            # Get the KEE annotations:
            if "KE-" in layer_name or "KEE-" in layer_name:
                # Loop over all KEEs or KEs:
                for label in layer.findall('.//label'):
                    start = int(label.attrib.get('start'))
                    end = int(label.attrib.get('end'))
                    if "KEE-" in layer_name:
                        kees.append(text[start:end])  # Read the KEE
                        kees_idx.append((start, end))  # Log the position of the KEE
                    else:
                        kes.append(text[start:end])  # Read the KE
                        kes_idx.append((start, end))  # Log the position of the KE
        
        sentences_data.append({
            'uid': uid,
            'constr_id': constr_id,
            'text': text,
            'kees': kees,
            'kees_idx': kees_idx,
            'kes': kes,
            'kes_idx': kes_idx
        })
    return sentences_data
    
sentence_list = []
xml_directory = '../data/constructicon/construction'

for filename in os.listdir(xml_directory):
    if filename.endswith('.xml'):
        xml_file = os.path.join(xml_directory, filename)
        data = parse_masked_sentences(xml_file)
        if data:
            sentence_list += data

sentences = pd.DataFrame.from_dict(sentence_list)
# sentences.set_index('uid', inplace=True)
sentences    

,uid,constr_id,text,kees,kees_idx,kes,kes_idx
0,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,[geschweige denn],"[(128, 143)]","[dass, nicht, jeder Sprecher und Führer , der ...","[(55, 59), (60, 65), (66, 110), (111, 125), (1..."
1,D86B38CFA5D2458D9F3615EDD55C9308DF423DB0,10,»Ohne Europa sind viele Fragen nicht mehr seri...,[geschweige denn],"[(72, 87)]","[»Ohne Europa sind viele Fragen, nicht, mehr s...","[(0, 30), (31, 36), (37, 48), (49, 69), (88, 97)]"
2,7B3FAB8B01ED862D3D3FA18531B0904BCF24DFF0,10,"Dies lässt sich damit begründen , dass vor und...",[geschweige denn],"[(126, 141)]",[dass vor und neben Alexander – soweit wir wis...,"[(34, 84), (85, 89), (90, 97), (98, 123), (142..."
3,B003E526D0AA301DA46458D32A82CEEFD7ADADA5,10,"Dennoch hat Hamas es nicht geschafft , ihre mi...",[geschweige denn],"[(83, 98)]","[Dennoch hat Hamas es, nicht, geschafft, ihre ...","[(0, 20), (21, 26), (27, 36), (39, 80), (99, 1..."
4,C3FFF1CA55CA8FF0A806C33E14A0FB331582698E,10,"Abermillionen rings um die Welt , die das Spek...",[geschweige denn],"[(183, 198)]","[Abermillionen rings um die Welt , die das Spe...","[(0, 92), (93, 98), (99, 180), (201, 246)]"
...,...,...,...,...,...,...,...
10263,B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,993,""" "" Aber da wird geredet und geredet und , sob...",[],[],[wird geredet],"[(12, 24)]"
10264,32E879397C3BA674F547058D7CB2B781C55FE1AC,993,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...",[],[],[wird getanzt],"[(105, 117)]"
10265,F0F280AA87B32AF6C4122E5B9C7906A31E88E555,993,"Hier wird getankt , und erst in der Morgenfrüh...",[],[],[wird getankt],"[(5, 17)]"
10266,549AF1E5C1904364885CE7F8DE324A9E6323629F,993,Jetzt wird gewartet,[],[],[wird gewartet],"[(6, 19)]"


In [9]:
sentences = sentences.explode(["kees", "kees_idx"], ignore_index=True)
sentences

,uid,constr_id,text,kees,kees_idx,kes,kes_idx
0,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)","[dass, nicht, jeder Sprecher und Führer , der ...","[(55, 59), (60, 65), (66, 110), (111, 125), (1..."
1,D86B38CFA5D2458D9F3615EDD55C9308DF423DB0,10,»Ohne Europa sind viele Fragen nicht mehr seri...,geschweige denn,"(72, 87)","[»Ohne Europa sind viele Fragen, nicht, mehr s...","[(0, 30), (31, 36), (37, 48), (49, 69), (88, 97)]"
2,7B3FAB8B01ED862D3D3FA18531B0904BCF24DFF0,10,"Dies lässt sich damit begründen , dass vor und...",geschweige denn,"(126, 141)",[dass vor und neben Alexander – soweit wir wis...,"[(34, 84), (85, 89), (90, 97), (98, 123), (142..."
3,B003E526D0AA301DA46458D32A82CEEFD7ADADA5,10,"Dennoch hat Hamas es nicht geschafft , ihre mi...",geschweige denn,"(83, 98)","[Dennoch hat Hamas es, nicht, geschafft, ihre ...","[(0, 20), (21, 26), (27, 36), (39, 80), (99, 1..."
4,C3FFF1CA55CA8FF0A806C33E14A0FB331582698E,10,"Abermillionen rings um die Welt , die das Spek...",geschweige denn,"(183, 198)","[Abermillionen rings um die Welt , die das Spe...","[(0, 92), (93, 98), (99, 180), (201, 246)]"
...,...,...,...,...,...,...,...
13112,B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,993,""" "" Aber da wird geredet und geredet und , sob...",NaN,NaN,[wird geredet],"[(12, 24)]"
13113,32E879397C3BA674F547058D7CB2B781C55FE1AC,993,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...",NaN,NaN,[wird getanzt],"[(105, 117)]"
13114,F0F280AA87B32AF6C4122E5B9C7906A31E88E555,993,"Hier wird getankt , und erst in der Morgenfrüh...",NaN,NaN,[wird getankt],"[(5, 17)]"
13115,549AF1E5C1904364885CE7F8DE324A9E6323629F,993,Jetzt wird gewartet,NaN,NaN,[wird gewartet],"[(6, 19)]"


In [10]:
sentences = sentences.explode(["kes", "kes_idx"], ignore_index=True)
sentences

,uid,constr_id,text,kees,kees_idx,kes,kes_idx
0,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)",dass,"(55, 59)"
1,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)",nicht,"(60, 65)"
2,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)","jeder Sprecher und Führer , der redet , auch","(66, 110)"
3,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)",etwas zu sagen,"(111, 125)"
4,57B9D1BEB68F44E7A51D1920F1EA6951E39469B5,10,Und dann ist da noch das generelle Problem mit...,geschweige denn,"(128, 143)",das letzte Wort,"(144, 159)"
...,...,...,...,...,...,...,...
31496,B5CB5D20EFEDAF65D460621CD3D554622B9EE1E9,993,""" "" Aber da wird geredet und geredet und , sob...",NaN,NaN,wird geredet,"(12, 24)"
31497,32E879397C3BA674F547058D7CB2B781C55FE1AC,993,"Das erfüllt einen mit Stolz "" "" , sagte Neid u...",NaN,NaN,wird getanzt,"(105, 117)"
31498,F0F280AA87B32AF6C4122E5B9C7906A31E88E555,993,"Hier wird getankt , und erst in der Morgenfrüh...",NaN,NaN,wird getankt,"(5, 17)"
31499,549AF1E5C1904364885CE7F8DE324A9E6323629F,993,Jetzt wird gewartet,NaN,NaN,wird gewartet,"(6, 19)"


In [11]:
with open("../out/CoMaPP_all.txt", "w") as file:
    file.write("")  # delete file content

# tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE")

json_comapp = []
csv_comapp = []
errors = 0
problematic_constructions = set()
unproblematic_constructions = set()


for _, row in tqdm(sentences.iterrows(), total=len(sentences)):
    tokenized = str(row["text"]).split()  # tokenizer.tokenize(...)
    
    try:
        ke_start, ke_end = row["kes_idx"]
        tokenized_kees = str(row["kees"]).split()  # tokenizer.tokenize(...)  # Split the KEEs if there are multi-word KEEs
    except TypeError:
        continue  # If there is nothing to mask or if there is no KEE, we can't use this example.
        
    masked_text = row["text"][:ke_start] + "<mask>" + row["text"][ke_end:]
    tokenized_masked = str(masked_text).split()
    # Rejoin "<", "mask" and ">".
    tokenized_masked = [
        '<mask>' if tokenized_masked[i] == '<' else tokenized_masked[i]  # add "<mask>" back in instead of "<"
        for i in range(len(tokenized_masked)) 
        if tokenized_masked[i] != 'mask' and tokenized_masked[i] != '>'  # remove "mask" and ">"
    ]
    
    try:
        kee_idx = [tokenized.index(tokenized_kee) for tokenized_kee in tokenized_kees]
        kee_query_idx = []
        for i in kee_idx:
            if tokenized_masked.index("<mask>") < i:  # indicates that the query_idx might have been shifted because the multi-token mask comes first
                offset = len(tokenized) - len(tokenized_masked)
                assert i - offset >= 0
                kee_query_idx.append(i - offset)
            else:
                kee_query_idx.append(i)
    except (ValueError, AssertionError) as e:
        print(row["constr_id"], type(e), e, "... Continuing ...")
        errors += 1
        problematic_constructions.add(row["constr_id"])
        continue
    
    unproblematic_constructions.add(row["constr_id"])
    
    out_json = [{
        "label": kee + str(row["constr_id"]),
        "target1": row["text"], 
        "target1_idx": idx, 
        "query": masked_text,
        "query_idx": q
    } for kee, idx, q in zip(tokenized_kees, kee_idx, kee_query_idx)]  # Split the KEEs if there are multi-word KEEs
    out_csv = [{
        "query": row["text"],
        "mask": row["text"][ke_start:ke_end],
        "ambigous_word": kee,
        "label": kee + str(row["constr_id"])  # This will be the new token that we will add to the LLM (named "<kee><i>" where <kee> and <i> are replaced by the KEE's name and i will be replaced by the construction it appears in).
    } for kee in tokenized_kees]
    
    json_comapp += out_json
    csv_comapp += out_csv
    
with open("../out/CoMaPP_all.json", "w") as file:
    json.dump(json_comapp, file, ensure_ascii=False)

f"{errors} elements from {len(problematic_constructions)} different constructions could not be saved as intended."

  0%|          | 0/31501 [00:00<?, ?it/s]

1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'> 'er' is not in list ... Continuing ...
1004 <class 'ValueError'>

'13012 elements from 230 different constructions could not be saved as intended.'

In [12]:
import csv

with open("./pseudowords/CoMapp_Dataset.csv", "w+", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["label", "query", "mask", "ambigous_word"])
    writer.writeheader()
    writer.writerows(csv_comapp)

In [13]:
# Load the pre-trained MBart-50 model and tokenizer
model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="de_DE", tgt_lang="de_DE")

In [14]:
# Define a masked input sequence and convert it into tokens
masked_sequence = "Ich bin <mask> gegangen."
input_ids = tokenizer.encode(masked_sequence, return_tensors="pt")

In [15]:
# Generate predictions
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=100, num_return_sequences=15, num_beams=20)

In [16]:
predicted_texts = []
for output in outputs:
    predicted_texts.append(tokenizer.decode(output, skip_special_tokens=True))
predicted_texts

['Ich bin nach Hause gegangen.',
 'Ich bin wieder nach Hause gegangen.',
 'Ich bin ins Bett gegangen.',
 'Ich bin wieder ins Bett gegangen.',
 'Ich bin in den Wald gegangen.',
 'Ich bin aus dem Haus gegangen.',
 'Ich bin ins Ausland gegangen.',
 'Ich bin ins Krankenhaus gegangen.',
 'Ich bin in eine andere Richtung gegangen.',
 'Ich bin in eine andere Welt gegangen.',
 'Ich bin gerade ins Bett gegangen.',
 'Ich bin in die Stadt gegangen.',
 'Ich bin heute früh gegangen.',
 'Ich bin wieder ins Leben gegangen.',
 'Ich bin nicht ins Bett gegangen.']

In [17]:
# Create a dataframe which matches the index of the constructions dataframe and the constr_id of the kees dataframe
kee_categories = sentences.join(constructions, on="constr_id")[["constr_id", "kees", "category", "name"]].drop_duplicates().reset_index(drop=True)
kee_categories

,constr_id,kees,category,name
0,10,geschweige denn,Negation,NEG_X_geschweige_denn_Y
1,10,Geschweige denn,Negation,NEG_X_geschweige_denn_Y
2,100,gleich,Äquativ_Plural,gleich_ADJ
3,1004,er,Superlativ_Klimax,ADJ1_ADJ1-er_NP
4,1006,st,Superlativ,PRÄP_ADJ-ster_NP
...,...,...,...,...
948,98,So,Äquativ,so_ADJ_wie_XP
949,980,NaN,Quantifizierung,CARD_KATEGORIE_NP
950,985,NaN,Relation_zweistellig,Transitiv_aktiv
951,99,er,Komparativ,ADJ-er_als_X


In [18]:
# Count the distinct names:
kee_categories["name"].value_counts()

name
Klammern                61
w-VL-Satz               54
es_gab_eine_Zeit_XP     26
wer                     19
IND.PRO_bis_auf_NP      16
                        ..
ähnlich_ADJ              1
Farbbez._in_Farbbez.     1
wie_ADJ                  1
Frage-Antwort            1
Intransitiv_passiv       1
Name: count, Length: 270, dtype: int64

Vergleiche wie ähnlich sich cased und uncased sind:

In [19]:
input_text_1 = "Wie Schulbücher den Mauerfall darstellen"
input_text_2 = "wie Schulbücher den Mauerfall darstellen"

input_tokens_1 = tokenizer(input_text_1, return_tensors="pt", padding=True, truncation=True)
input_tokens_2 = tokenizer(input_text_2, return_tensors="pt", padding=True, truncation=True)

torch.stack((input_tokens_1.input_ids, input_tokens_2.input_ids))

tensor([[[250003,   4887,  27325, 206709,    168,  23788,     56,   9146,
            1011,  17929,      2]],

        [[250003,   1225,  27325, 206709,    168,  23788,     56,   9146,
            1011,  17929,      2]]])

In [20]:
with torch.no_grad():
    model_output_1 = model(**input_tokens_1)
    model_output_2 = model(**input_tokens_2)
    
hidden_states_1 = model_output_1.encoder_last_hidden_state
hidden_states_2 = model_output_2.encoder_last_hidden_state

sequence_emb_1 = hidden_states_1[:, 5, :]
sequence_emb_2 = hidden_states_2[:, 5, :]

sequence_emb_1, sequence_emb_2

(tensor([[-0.2712,  0.2646,  0.1158,  ..., -0.3121, -0.0238,  0.2057]]),
 tensor([[-0.3570,  0.2023,  0.1094,  ..., -0.2422, -0.1210,  0.2481]]))

In [21]:
# TODO Markiere die zu desambiguierenden Wörter
# Position ergibt keinen Sinn, weil die maskierten Bereiche beliebig lang sein können (und damit der Index variabel ist in Abhängigkeit der länge der generierten Phrase.
# Idee wäre festgelegtes Symbol (z. B. §) oder gar kein Symbol und einfach heuristisch von .index(wort) ausgehen.

In [22]:
from transformers import Text2TextGenerationPipeline

# Create a pipeline for text generation with the "fill-mask" task
generator = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)

# Define a masked input sequence
masked_sequence = "Ich bin <mask> gegangen."

# Generate predictions
predicted_texts = generator(masked_sequence, max_length=100, num_return_sequences=15, num_beams=20)

# Extract the predicted texts from the results
predicted_texts = [result['generated_text'] for result in predicted_texts]

# Print the predicted texts
print(predicted_texts)

['Ich bin nach Hause gegangen.', 'Ich bin wieder nach Hause gegangen.', 'Ich bin ins Bett gegangen.', 'Ich bin wieder ins Bett gegangen.', 'Ich bin in den Wald gegangen.', 'Ich bin aus dem Haus gegangen.', 'Ich bin ins Ausland gegangen.', 'Ich bin ins Krankenhaus gegangen.', 'Ich bin in eine andere Richtung gegangen.', 'Ich bin in eine andere Welt gegangen.', 'Ich bin gerade ins Bett gegangen.', 'Ich bin in die Stadt gegangen.', 'Ich bin heute früh gegangen.', 'Ich bin wieder ins Leben gegangen.', 'Ich bin nicht ins Bett gegangen.']
